In [ ]:
from urllib import request, parse
import os
import sys
from datetime import datetime
from functools import partial
from collections import defaultdict
import time
from datetime import datetime
import re
import zipfile
# from multiprocessing import Pool

import pandas as pd
import matplotlib as mpl
import numpy as np
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
os.chdir(sys.path[0])

In [ ]:
ev_df = pd.read_csv(os.path.join('data', 'results', 'early_voting.csv'), low_memory = False)

In [ ]:
ed_df = pd.read_csv(os.path.join('data', 'results', 'election_day.csv'), low_memory = False)

In [ ]:
relevant_elections = [
    # '2024 NOVEMBER 5TH GENERAL ELECTION',
    '2024 MARCH 5TH DEMOCRATIC PRIMARY',
    '2024 MARCH 5TH REPUBLICAN PRIMARY',
    # '2022 NOVEMBER 8TH GENERAL ELECTION',
    # '2022 MAY 24TH REPUBLICAN PRIMARY RUNOFF',
    # '2022 MAY 24TH DEMOCRATIC PRIMARY RUNOFF',
    # '2022 MARCH 1ST REPUBLICAN PRIMARY',
    # '2022 MARCH 1ST DEMOCRATIC PRIMARY',
    '2020 NOVEMBER 3RD GENERAL ELECTION',
    '2020 JULY 14TH REPUBLICAN PRIMARY RUNOFF',
    '2020 JULY 14TH DEMOCRATIC PRIMARY RUNOFF',
    '2020 MARCH 3RD DEMOCRATIC PRIMARY',
    '2020 MARCH 3RD REPUBLICAN PRIMARY',
]

In [ ]:
ev_df = ev_df[ev_df['election_name'].str.strip().isin(relevant_elections)]
ed_df = ed_df[ed_df['election_name'].str.strip().isin(relevant_elections)]

In [ ]:
ev_df['timing'] = 'EV'
ed_df['timing'] = 'ED'
ed_df['VOTING_METHOD'] = 'IN-PERSON'

In [ ]:
ev_df.head()

In [ ]:
ed_df.head()

In [ ]:
df = pd.concat([ev_df, ed_df])[['COUNTY', 'VOTING_METHOD', 'PRECINCT', 'date', 'election_id', 'election_name', 'timing', 'VOTER COUNT']] \
        .groupby(['COUNTY', 'VOTING_METHOD', 'PRECINCT', 'election_id', 'election_name', 'timing']) \
        .agg({
            'date': 'max',
            'VOTER COUNT': 'sum'
        }).reset_index()

In [ ]:
counties = [
    'collin',
    'dallas',
    'denton',
    'ellis',
    'hopkins',
    'hunt',
    'johnson',
    'kaufman',
    'rockwall',
    'tarrant'
]

In [ ]:
dfw_df = df[df['COUNTY'].str.lower().isin(counties)]

In [ ]:
voters_df = pd.read_csv(os.path.join('data', 'registration', 'voter_records_full.csv'), sep = chr(29), low_memory = False)

In [ ]:
voters_df['status_code'].unique()

In [ ]:
dfw_voters_df = voters_df[voters_df['county'].str.lower().isin(counties)]

In [ ]:
dfw_n_voters_df = dfw_voters_df[['county', 'precinct', 'edr']].groupby(['county', 'precinct', 'edr']).size().reset_index().rename(columns = {0: 'registered voters'})

In [ ]:
dfw_n_voters_df['county'] = dfw_n_voters_df['county'].str.upper()

In [ ]:
dfw_n_voters_df[(dfw_n_voters_df['edr'] <= '2022-01-01') & (dfw_n_voters_df['county'] == 'COLLIN')]['registered voters'].sum()

In [ ]:
# print(dfw_voters_df_group[(dfw_voters_df_group['edr'] <= '2020-12-31') & (dfw_voters_df_group['county'] == 'COLLIN')]['registered voters'].sum())

# dfw_df.loc[:,'voters'] = pd.NA

# for _, row in dfw_df.iterrows():
#     voters = dfw_voters_df_group[(dfw_voters_df_group['edr'] <= row['date']) & (dfw_voters_df_group['county'] == row['COUNTY'])]['registered voters'].sum()
#     print(row['COUNTY'], row['date'], voters)

In [ ]:
turnout_precinct_dfw_df = dfw_df[['COUNTY', 'PRECINCT', 'VOTER COUNT', 'election_name', 'timing']].groupby(['COUNTY', 'PRECINCT', 'timing', 'election_name']).agg({
    'VOTER COUNT': 'sum'
}).reset_index()

In [ ]:
turnout_precinct_dfw_df.set_index(['COUNTY', 'PRECINCT']).to_csv('turnout_precinct_dfw.csv')

In [ ]:
turnout_county_dfw_df = dfw_df[['COUNTY', 'VOTER COUNT', 'election_name', 'timing']].groupby(['COUNTY', 'timing', 'election_name']).agg({
    'VOTER COUNT': 'sum'
}).reset_index()

In [ ]:
turnout_county_dfw_df.set_index('COUNTY').to_csv('turnout_county_dfw.csv')